In [1]:
!pip install omegaconf==2.3.0
!pip install ultralytics==8.1.45

In [2]:
#nvgpu==0.9.0
#pynvml==11.4.1

In [3]:
from src import inference as inf
import boto3
import json
model_dir = ""
model = inf.model_fn(model_dir)
#####################################################
s3_bucket = "PUT HERE BUCKET NAME IN S3"
s3_key = "PUT HERE PATH TO IMAGE IN S3"
image_file_name = "frame226.jpg"
local_dir = 'input'
payload_tuple = (s3_bucket, s3_key, image_file_name, local_dir)
payload = "|".join(payload_tuple).encode("utf-8")
#####################################################
content_type = 'text/csv'
input_data = inf.input_fn(payload, content_type)
prediction_output = inf.predict_fn(input_data, model)
str_result = inf.output_fn(prediction_output, content_type)
result = json.loads(str_result)
print("status: ", result["status"])
print("boxes: ", result["boxes"])

Executing model_fn from inference.py ...
request_body afre decode:  content-recognition-ci|sagemaker/FaceClusterDataset/FaceClusteringDataset/00c28dbb-2ba2-4625-815d-1d64849c7f7e/|frame226.jpg|input
Executing predict_fn from inference.py ...

0: 640x640 1 face, 5.4ms
Speed: 5.0ms preprocess, 5.4ms inference, 780.9ms postprocess per image at shape (1, 3, 640, 640)
Executing output_fn from inference.py ...
status:  !!! The directory input) does exist: True| The file input/frame226.jpg does exist: True
boxes:  [[109.56486511230469, 27.41848373413086, 184.64451599121094, 175.40426635742188, 0.3425535261631012, 0.0]]


In [4]:
import os, sagemaker, subprocess, boto3
from datetime import datetime
from sagemaker import s3
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel
from sagemaker.deserializers import JSONDeserializer

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [5]:
!pwd

/home/ec2-user/SageMaker/yolov8_face_detection_input_s3_async


In [6]:
#create the model artifacts 
bashCommand = "tar -czf  model.tar.gz src model"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

In [7]:
role = get_execution_role()
sess = sagemaker.Session()

model_name = "yolov8n-face.pt"
model_data = 'model.tar.gz'
model = PyTorchModel(entry_point='inference.py',
                     model_data=model_data, 
                     framework_version='1.12', 
                     py_version='py38',
                     role=role,
                     source_dir = "src",
                     env={'TS_MAX_RESPONSE_SIZE':'20000000', 'YOLOV8_MODEL': model_name},
                     sagemaker_session=sess)

In [8]:
ENDPOINT_NAME = 'yolov8-pytorch-facedetection-ADOLIA' + str(datetime.utcnow().strftime('%Y-%m-%d-%H-%M-%S-%f'))

# Store the endpoint name in the history to be accessed by 2_TestEndpoint.ipynb notebook
# We dnt have _2_TestEndpoint.ipynb in this project
%store ENDPOINT_NAME

Stored 'ENDPOINT_NAME' (str)


In [9]:
ENDPOINT_NAME

'yolov8-pytorch-facedetection-ADOLIA2024-06-12-14-10-14-954785'

In [10]:
from sagemaker.async_inference.async_inference_config import AsyncInferenceConfig

# https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html
INSTANCE_TYPE = 'ml.g5.xlarge'
print("ENDPOINT_NAME: ", ENDPOINT_NAME)
print("INSTANCE_TYPE: ", INSTANCE_TYPE)

from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

bucket = "PUT HERE BUCKET NAME IN S3"
prefix = "PUT HERE LOCATION PATH"

# Create an empty AsyncInferenceConfig object to use default values
async_config = AsyncInferenceConfig(output_path = f"s3://{bucket}/{prefix}/output")

# deploy model to SageMaker Inference
predictor = model.deploy(async_inference_config = async_config,
                         initial_instance_count = 1,
                         instance_type          = INSTANCE_TYPE,
                         #serializer             = CSVSerializer,
                         #deserializer           = JSONDeserializer(),
                         endpoint_name          = ENDPOINT_NAME
)

ENDPOINT_NAME:  yolov8-pytorch-facedetection-ADOLIA2024-06-12-14-10-14-954785
INSTANCE_TYPE:  ml.g5.xlarge
---------!

In [11]:
ENDPOINT_NAME

'yolov8-pytorch-facedetection-ADOLIA2024-06-12-14-10-14-954785'

# Invoke Endpoint: Face Detection

In [12]:
import time
#%store -r ENDPOINT_NAME
sm_client = boto3.client(service_name="sagemaker")

# Restore the endpoint name stored in the 2_DeployEndpoint.ipynb notebook
print(f'Endpoint Name: {ENDPOINT_NAME}')

endpoint_created = False
while True:
    response = sm_client.list_endpoints()
    for ep in response['Endpoints']:
        print(f"Endpoint Status = {ep['EndpointStatus']}")
        if ep['EndpointName']==ENDPOINT_NAME and ep['EndpointStatus']=='InService':
            endpoint_created = True
            break
    if endpoint_created:
        break
    time.sleep(5)

Endpoint Name: yolov8-pytorch-facedetection-ADOLIA2024-06-12-14-10-14-954785
Endpoint Status = InService


In [ ]:
#####################################################
s3_bucket = "PUT HERE BUCKET NAME IN S3"
s3_key = "PUT HERE PATH TO S3"
image_file_name = "frame226.jpg"
# it does not work if you define local_dir = "input", it has to be local_dir = '/opt/ml/input'
local_dir = '/opt/ml/input'
payload_tuple = (s3_bucket, s3_key, image_file_name, local_dir)
payload = "|".join(payload_tuple)
#####################################################
s3_client = boto3.client('s3')
sm_session = sagemaker.session.Session()

bucket_name = "PUT HERE BUCKET NAME IN S3" # sm_session.default_bucket()
file_name = "payload/yolov8-face-detection/" + image_file_name.split(".")[0] + ".json"

# Upload the JSON string as a file to S3
s3_client.put_object(Body=payload, Bucket=bucket_name, Key=file_name)
s3_payload_location = f"s3://{bucket_name}/{file_name}"
print(f"File saved to {s3_payload_location}")

# Asynchronous Inference. <br>
There are two methods that give the same results:<br>
1) using predict_async (please set isPredict = True); and <br>
2) invoke_endpoint_async (set isPredict = False).


In [17]:
isPredict = False
if isPredict:
   res = predictor.predict_async(input_path=s3_payload_location)
   res_s3_output_location = res.output_path
   res_s3_error_location11  = res.failure_path 
   print(f"AWS S3 uploaded inference responses location: {res_s3_output_location11}")
   print(f"AWS S3 failed requests errors : {res_s3_error_location}")
else:
   print("ENDPOINT_NAME: ", ENDPOINT_NAME)
   print("s3_payload_location: ", s3_payload_location)
   boto_session = boto3.session.Session()
   sm_runtime = boto_session.client("sagemaker-runtime")
   response = sm_runtime.invoke_endpoint_async(
     EndpointName = ENDPOINT_NAME, 
     InputLocation= s3_payload_location
   )
   res_s3_output_location = response["OutputLocation"]
   print(f"OutputLocation: {res_s3_output_location}") 

In [18]:
if not isPredict:
   print("ENDPOINT_NAME: ", ENDPOINT_NAME)
   print("s3_payload_location: ", s3_payload_location)
   boto_session = boto3.session.Session()
   sm_runtime = boto_session.client("sagemaker-runtime")
   response = sm_runtime.invoke_endpoint_async(
     EndpointName = ENDPOINT_NAME, 
     InputLocation= s3_payload_location
   )
   res_s3_output_location = response["OutputLocation"]
   print(f"OutputLocation: {res_s3_output_location}")

ENDPOINT_NAME:  yolov8-pytorch-facedetection-ADOLIA2024-06-12-14-10-14-954785
s3_payload_location:  s3://content-recognition-models/payload/yolov8-face-detection/frame226.json
OutputLocation: s3://content-recognition-models/face-tracker-response/output/9a4f2bab-7658-42ca-956b-dfebe77d80d0.out


In [19]:
import urllib, time
from botocore.exceptions import ClientError


def get_output(output_location):
    output_url = urllib.parse.urlparse(output_location)
    bucket = output_url.netloc
    key = output_url.path[1:]
    while True:
        try:
            return sm_session.read_s3_file(bucket=output_url.netloc, key_prefix=output_url.path[1:])
        except ClientError as e:
            if e.response["Error"]["Code"] == "NoSuchKey":
                print("waiting for output...")
                time.sleep(2)
                continue
            raise

In [20]:
result_ad = get_output(res_s3_output_location)
result_ad = json.loads(result_ad)
status = result_ad["status"]
boxes =  result_ad["boxes"]
print(f"Output status: {status}")
print(f"Output boxes: {boxes}")
orig_image = result_ad["orig_image"].copy()

Output status: !!! The directory /opt/ml/input) does exist: True| The file /opt/ml/input/frame226.jpg does exist: True
Output boxes: [[109.56016540527344, 27.404191970825195, 184.64613342285156, 175.42283630371094, 0.3421856760978699, 0.0]]


In [ ]:
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt

# original image that I download from s3 has already bounding box (largest bounding box) and
# face detector add another bounding box (smaller box)

orig_image = np.array(result_ad["orig_image"]).astype(np.uint8)

image_height, image_width, _ = orig_image.shape
model_height, model_width = 300, 300
x_ratio = image_width/model_width
y_ratio = image_height/model_height

if 'boxes' in result_ad:
    for idx,(x1,y1,x2,y2,conf,lbl) in enumerate(result_ad['boxes']):
        print("x1,y1,x2,y2: ", x1,y1,x2,y2)
        # Draw Bounding Boxes
        x1, x2 = int(x_ratio*x1), int(x_ratio*x2)
        y1, y2 = int(y_ratio*y1), int(y_ratio*y2)
        color = (random.randint(10,255), random.randint(10,255), random.randint(10,255))
        cv2.rectangle(orig_image, (x1,y1), (x2,y2), color, 4)
        print("left top point: ", (x1,y1), ", right bottom point: ", (x2,y2))
        print("detected object class: ", lbl)
        print("detection confidence: ", conf)
        cv2.putText(orig_image, f"Class: {int(lbl)}", (x1,y1-40), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
        cv2.putText(orig_image, f"Conf: {int(conf*100)}", (x1,y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)

plt.imshow(cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
predictor.delete_model()
predictor.delete_endpoint() 

To make a compressed tar ball of the current directory <br>
tar -czvf yolov8_face_detection_input_s3_realtime.tar.gz .  <br>
Unzip: <br>
tar -xzvf yolov8_face_detection_input_s3_realtime.tar.gz